In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [5]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lendingDF = pd.read_csv("lending_data.csv")

# Review the DataFrame
lendingDF.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [7]:
# Separate the data into labels and features

# Separate the y variable, the labels
y=lendingDF["loan_status"]

# Separate the X variable, the features
X=lendingDF.drop(columns="loan_status")

In [8]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [9]:
# Review the X variable DataFrame
X[:5]

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [11]:
# Check the balance of our target values
lendingDF["loan_status"].value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [12]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [13]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifierModel = LogisticRegression(solver='lbfgs', random_state=1)
classifierModel
# Fit the model using training data
classifierModel.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [15]:
# Make a prediction using the testing data
predictions = classifierModel.predict(X_test)

#dataframe w/the acutals vs predicted values
pd.DataFrame(
    {
        "Actual":y_test,
        "Predictions": predictions
    }
).head()

,Actual,Predictions
36831,0,0
75818,1,0
36563,0,0
13237,0,0
43292,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [16]:
# Print the balanced_accuracy score of the model
classifierModel.score(X_train, y_train)

0.9914878250103177

In [17]:
# Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix

# use the test data and the predictions to generate the confusion matrix
confusion_matrix(y_test, predictions)

array([[18679,    80],
       [   67,   558]])

In [18]:
# Print the classification report for the model
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions, target_names=["Purple", "Yellow"]))

              precision    recall  f1-score   support

      Purple       1.00      1.00      1.00     18759
      Yellow       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model shows that there is about a 99% accuracy of the predictions.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [19]:
pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversample
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)


In [24]:
# Count the distinct values of the resampled labels data
y_resampled

0         1
1         0
2         0
3         1
4         1
         ..
120289    0
120290    0
120291    0
120292    0
120293    0
Name: loan_status, Length: 120294, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [25]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifierModel = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=1)

classifierModel.fit(X_train, y_train)

# Make a prediction using the testing data
y_pred = classifierModel.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [27]:
# Print the balanced_accuracy score of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9945550521634315


In [29]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, y_pred)

array([[12054,    70],
       [   61, 11874]])

In [30]:
# Print the classification report for the model
print(classification_report(y_test, y_pred, target_names=["Purple", "Yellow"]))

              precision    recall  f1-score   support

      Purple       0.99      0.99      0.99     12124
      Yellow       0.99      0.99      0.99     11935

    accuracy                           0.99     24059
   macro avg       0.99      0.99      0.99     24059
weighted avg       0.99      0.99      0.99     24059



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** It steadies it to a more consistent 99% accuracy.